### Note
Before running the script, make sure to download Selenium Chrome Webdriver from [here](https://chromedriver.chromium.org/downloads) to your custom local directory.  

In [ ]:
import time
from datetime import *
import pandas as pd
import numpy as np

In [ ]:
import requests, re, typing
from bs4 import BeautifulSoup as soup
import contextlib
def _remove(d:list) -> list:
   return list(filter(None, [re.sub('\xa0', '', b) for b in d]))

@contextlib.contextmanager
def get_weather_data(url:str, by_url = True) -> typing.Generator[dict, None, None]:
   d = soup(requests.get(url).text if by_url else url, 'html.parser')
   _table = d.find('table', {'id':'wt-his'})
   _data = [[[i.text for i in c.find_all('th')], *[i.text for i in c.find_all('td')]] for c in _table.find_all('tr')]
   [h1], [h2], *data, _ = _data
   _h2 = _remove(h2)
   yield {tuple(_remove(h1)):[dict(zip(_h2, _remove([a, *i]))) for [[a], *i] in data]}

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Direct to Selenium Chrome Webdriver
driver = webdriver.Chrome("/your/webdriver/directory")

_d = {}

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

for m in range(1, len(months)+1):
    driver.get("https://www.timeanddate.com/weather/usa/new-york/historic?month="+str(m)+"&year=2017")
    options = driver.find_element_by_id('wt-his-select').find_elements_by_tag_name('option')
    for i in range(1, len(options)+1):
        options[i-1].click()
        WebDriverWait(driver, 10).until(
            EC.text_to_be_present_in_element((By.ID, "wt-his-title"), "New York Weather History for "+str(i)+" "+months[m-1]+" 2017")
        )
        print(driver.find_element_by_id("wt-his-title").text, "New York Weather History for "+str(i)+" "+months[m-1]+" 2017")
        with get_weather_data(driver.page_source, False) as weather:
            _d[options[i-1].text] = weather
driver.close()

In [ ]:
for a, b in _d.items():
    print(a, b)